# Rust 101

## Ziel
Das Ziel des Tutorials ist es, eine Einführung in Rust zu bekommen, und am Ende einen Compiler und Interpreter von UB++ zu haben. Das ganze Projekt wird am Ende als Github-Repository zur Verfügung gestellt. Hier werden wir Schritt für Schritt versuchen, uns an das Endprodukt anzutasten, und jeweils einzelne wichtige Aspekte von Rust kennen lernen.

## UB++

Wie wir alle wissen, machen wir Geilen Scheiss (TM). Es ist aber viel geilerer Scheiss, wenn der Scheiss auch so geschrieben wird, wie der Schnabel wächst. Entsprechend kann jeder in diesem Tutorial, die Keywords seinem Schnabel nach anpassen.

UB++ ist eine einfache (interpretierte) Sprache, angelehnt an Java, die simple Logik und Berechnungen erlaubt.

```
definier e variable wo priimZahl heisst mit em wert e frog "Was füre Zahl wotsch teste?";
definier e variable wo teiler heisst mit em wert 2;
definier e variable wo primzahlWurzel heisst mit em wert priimZahl hoch 0.5;
definier e variable wo rest heisst mit em wert -1;
definier e variable wo priimZahlGfunde heisst mit em wert wohr;

solang de teiler kliiner oder gliich isch wie primzahlWurzel mach {
    falls de (priimZahl rest teiler) isch gliich wie 0 mach {
        priimZahlGfunde isch falsch;
        stop;
    };
    teiler isch teiler plus 1;
}

falls d priimZahlGfunde mach {
    gib us "S isch e priim zahl";
} suscht {
    gib us "S isch kei priim zahl";
};
```

## Rust installieren

Am einfachsten wird Rust direkt mit dem Bash-Skript von der [Rust-Seite](https://rustup.rs) installiert:

```
curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh
```

Der Installer sollte durch die Installation führen, wobei die defaults alle in Ordnung sein sollten. 

Sollte Rust schon installiert sein, bietet es sich an, Rust mit folgendem Befehl upzudaten:

```bash
 rustup update
```

## Jupyter installieren

Um dieses Notebook zu benutzen, braucht es [Jupyter](https://jupyter.org/install).

## Rust Jupyter Kernel installieren

Abschliessend wird der Rust-Kernel für die Jupyter-Notebooks benötigt. Dieser kann vie cargo wie folgt installiert werden:

```bash
cargo install evcxr_jupyter 
```
## Cargo

Cargo ist das `all-in-one` Tool der Rust-Entwicklung. 

- `cargo new [--lib] <name>`: Erstellt ein neues Rust-Projekt mit Name "name"
- `cargo build [--release]`: Buildet das Rust-Projekt im aktuellen Ordner (sucht nach einem Cargo.toml)
- `cargo run -- [programm arguments]`: Lässt das Rust-Projekt kompillieren und startet es mit den entsprechenden Argumenten
- `cargo add <dependency>`: Fügt eine Dependency zum Cargo.toml hinzu

### Cargo.toml
Das Cargo.toml beinhaltet wichtige Informationen zum Projekt, wie z.B. die Build-Products, Meta-Daten wie Author, Name usw. aber auch alle Abhängigkeiten. Es gibt viele verschiedene Mölgichkeiten Dependencies anzugeben, z.B. direkt vom lokalen File-System, von einem Git-Repository oder aber von [crates.io](https://crates.io).


<!--
 Copyright (c) 2022 Ubique Innovation AG <https://www.ubique.ch>
 
 This Source Code Form is subject to the terms of the Mozilla Public
 License, v. 2.0. If a copy of the MPL was not distributed with this
 file, You can obtain one at http://mozilla.org/MPL/2.0/.
-->

## Abhängigkeiten

Wir werden in diesem Tutorial [`pest`](https://pest.rs/) benutzen. `pest` bietet uns eine Möglichkeit, die Grammatik für unsere Sprache in einer externen Datei auszulagern, und dann bereits die einzelnen Tokens als Objekte zu erstellen.

Um in Rust Abhängikeiten hinzuzufügen, müssen wir (normalerweise) mit `cargo add <name_der_dependency>` im Rust-Projektordner die Abhängigkeit hinzufügen. Natürlich kann dies auch manuell im `Cargo.toml` gemacht werden. 

Da wir hier aber Jupyter-Notebooks verwenden, müssen wir uns nicht direkt mit dem auseinandersetzen. Folgende zwei Abhängigkeiten werden wir in unserem Projekt benutzen entsprechend müssen wir die unserem Jupyter-Kernel hinzufügen.



In [ ]:
:dep pest = "*"
:dep pest_derive = "*"

## Code zum Notebook

Der vollständige Code befindet sich unter `ubpp/src` und ist in drei Steps eingeteilt, die ungefähr der Einführungsreihenfolge im Notebook entsprechen. Falls ein Schritt nicht funktioniert, kann der Code kopiert werden. Er sollte allerdings nur als Hilfe dienen ;).

## Boilerplate

Um nun einen Einstieg für unsere kleine Script-Sprache zu bieten, lassen wir `pest` alle Implementierungen anhand unseres Grammar-Files ableiten (`#[derive]`).

Mit dem `#[grammar]`-Attribut können wir den Pfad zu unserem Grammar-File angeben. In unserem Fall hier, geben wir den absoluten Pfad an, damit das Grammar-File gefunden wird, unabhängig von wo aus unser Rust-Programm kompiliert wird.

Für das Notebook wird, anders als im Code-Beispiel, alles in einer Datei gemacht. Aufspalten zu verschiedenen Modulen etc. ist nicht relevant für dieses Projekt.

In [ ]:
use pest::Parser;
use pest::iterators::Pairs;

// Wird für den Jupyter-Kernel gebraucht, mit den neueren Rust-Versionen ist dies allerdings hinfällig.
extern crate pest;
#[macro_use]
extern crate pest_derive;


#[derive(Parser)]
#[grammar = "/Users/patrickamrein/Documents/Ubique/git/introduction-to-rust/ubpp.pest"]
pub struct UBPP;

## Erste Schritte

Da wir nun erstmals die Grammatik unserer Sprache geladen haben, können wir nun schauen, wie unser Code-Beispiel von Oben aussieht. Wir bekommen von  `pest` ein geparsedes File, das schon "Tokenized" ist, und nur  drauf wartet von uns umgewandelt zu werden.

Wir sehen unten ein Beispiel von einem `Multiline-Raw-String-Literal`, das es uns erlaubt, einen String über mehrere Zeilen zu schreiben, aber auch Anführungszeichen ohne escaping zu benutzen. Strings in Rust kann man grundsätzlich immer über mehrere Zeilen schreiben.

Danke dem "derive" Attribut auf unserer Klasse (struct UBPP), wurde vom Rust-Compiler anhand des `Pest-Files` bereits eine Menge von Implementierungen erstellt. Wird können nun diese "statische" Funktion auf der Struct mit folgendem Syntax aufrufen:
```rust
UBPP::parse(Rule::file, primzahl_tester).unwrap();
```

Beachte, dass solche assoziierte Funktionen auch auf Instanzen aufgerufen werden können. Wir werden später sehen, wie wir Funktionen einer Klasse hinzufügen können. Als erstes soll hier angemerkt werden, dass folgender Syntax equivalent ist:

In [ ]:
struct Test {}
impl Test {
    fn test_function(&self) {
        println!("test");
    }
}

let test_instance = Test{};
Test::test_function(&test_instance);
test_instance.test_function();

In [ ]:
let primzahl_tester = r#"
definier e variable wo priimZahl heisst mit em wert e frog "Was füre Zahl wotsch teste?";
"#;
let parse_result = UBPP::parse(Rule::file, primzahl_tester).unwrap();

Wir sehen hier, das "unwrap", das benutzt wird. Die `parse` Funktion liefert uns nämlich ein `Result`-Objekt zurück, auf welchem wir keine weiteren Opperation des eigentlichen Typen ausführen können. Um an das innere Objekt zu kommen, müssten wir ein Error-Handling einbauen (sehen wir später), oder wir wählen wie hier den einfachen (und während der Entwicklung durchaus legtimen) Weg fes force-unwrap.

Doch wie können wir jetzt sehen, was geparsed wurde? Rust kennt zwei verschiedene "Anzeige" Arten. Beide der Anzeige-Arten werden als "Traits" implementiert (ähnlich wie Interfaces resp. Protocols oder wie sie sonst heissen), und können für jeden Typen implementiert werden.

Zum einen wäre das Trait `Display`, welches am Ende eine `to_string` Funktion zur Verfügung stellt. Dies kann benutzt werden um eine Standard-Darstellung eines Typens zu definieren. 

Die zweite Möglichkeit ist  `Debug` und ermöglicht eine erweiterte Darstellung eines Typen für Debugging-Zwecke. Das `Debug`-Trait kann ebenfalls für jeden Typen "derived" werden, was dazuführt, dass jedes Feld mit Name und Wert angezeigt wird.

Wir vergleichen nun mal die zwei Arten mit dem Parse-Result.

Für die Ausgabe verwenden wir das `println`-Macro, welches uns "Compile-Time-Format-Strings" gibt. zwei geschweifte Klammern werden benutzt um die `to_string` Funktion eines Typs zu benutzen. Mit ":?" innerhalb der Klammern können wir Rust auffordern, die Debug-Implementierung aufzurufen. Wenn man zusätzlich noch einen "#" vor das Fragezeichen setzt, vordert man einen `pretty-debug-print`.

Es gibt noch weitere Format-String modifiers, auf welche wir hier jetzt nicht weiter eingehen.

In [ ]:
println!("{:?}", parse_result);
// println!("{:#?}", parse_result);
// println!("{}", parse_result);

## Das AST-Model (Step1)

Jetzt wo wir eine von `pest` geparsed Struktur haben, wollen wir unsere eigene Struktur aufbauen, die anschliessend relativ einfach ausgeführt werden kann (Step2). Wir nennen diese Struktur AST (Abstract Syntax Tree) und sie representiert unser Program im Speicher.

Wir werden nun versuchen, nach und nach die verschiedenen Klassen (Structs) und Enums aufzubauen. Wir fangen mit dem tiefsten Element an (Atomics) und bauen immer komplexere Strukturen, die aus diesen Bauteilen zusammengesetzt sind.

> Wir werden jetzt verschiedene Enums/Structs und so weiter einfügen. Da wir mit selbstreferenzierten Typen arbeiten musst du mit Code ausführen warten, bis wir alle Teile beisamen haben.
## Expressions
### Atomics

In unserer Sprache wird es fünf verschiedene "Atomic"-Datatypes geben. Wir werden mit alle Zahlen-Typen als "double" (64bit Floating-Point) darstellen, was das ganze Handling vereinfacht, und für unseren Use-Case ausreichend ist. Wir nennen diesen Datentypen `Number`. Weiter wird es einen `String`, einen `Bool`, einen `Null` und einen `Interrupt` Atomic geben.

Der `Interrupt` wird ein Atomic sein, um ein frühzeiteges Ende einer While-Schlaufe erkennen zu können (und ist etwas ein Hack, aber who cares, wir sind ja die Language-Architects).

Wir werden viele der Typen als Rust-Enums handeln. Der Rust-Enum Typ ist mehr als eine blosse Enumeration, sondern kann auch komplexe Objekte halten.

Doch vorerst wollen wir kurz auf die verschiedenen Grund-Datentypen von Rust eingehen:

- `u8/u16/u32/u64 | i8/i16/i32/i64`: Dies sind die Integer Datentypen in Rust. Der Buchstabe gibt jeweils an, ob der Typ "*u*nsigned" ist oder signed. Die Zahl gibt die jeweilige Bit-Breite an, also für ein Byte zum Beispiel "u/i8". Je nach Platform wird bis `i/u128` unterstützt.

- `f32/f64`: Sind die Fliesskommadatentypen. Die Zahl stellt wiederum die Bitbreite an, also f32 für single Precision und f64 für double precision.

- `bool`: Der übliche Boolean-Datentyp.

- `String`: Ist ein Heap allozierter veränderbarer String-Typ. Strings in Rust _müssen_ immer gültiges UTF-8 sein.

- `&str`: Ist ein "geliehener" String, respektive "String-Slice". Wir wollen hier vorerst nicht genauer auf Borrows resp. den Unterschied von `String` und `&str` eingehen.

Wir haben hier bewusst auf komplexere Typen wie Arrays, Maps, Synchronisations-Typen und ähnliches verzichtet. Im verlaufe des Tutorials werden wir uns noch mit Arrays und Maps auseinandersetzen.

Es gibt drei verschiedene Typen von Enum-Variants:

- Unit-Type: Der Unit-Type ist sehr ähnlich zum klassischen C enum Typ. Er stellt einfach eine "Zahl" oder Enumeration dar

- Tuple-Type: Der Tuple-Type ist, anders als der Unit-Typ, fähig ein oder mehrere Objekte zu halten. Diese werden einfach als Tuple übergeben, und haben an sich keinen Namen, sondern bloss eine festgelegte Reihenfolge

- Struct-Type: Struct-Type hält eine Struct (also eine Rust-Klasse) und hat benannte Felder.


Folgend sehen wir das Gerüst für unseren Atomic-Datatype. Es fehlt allerdings noch der Boolean Typ. Füge ihn hinzu.

In [ ]:
#[derive(Debug, Clone)]
pub enum Atomic {
    String(String),
    Number(f64),
    /* Füge hier noch einen weiteren Typen, nämlich den Boolean Typ ein */
    Null,
    Interrupt
}


> Wir sehen das Attribut `#[derive(Debug, Clone)]` über dem Enum. Was bedeutet das? Wir sehen hier das "derive" Attribut. Dieses Attribut dient dazu, default Implementierungen für bestimmte Typen zur Verfügung zu stellen. Dieses Derive Macro ist ein sogenanntes "Procedural Macro" und erlaubt es Rust-Code während dem resp. vor dem Kompilieren auszuführen. In unserem Falle werden Implementierungen für den Debug-Output und für die "Deep-Copy" erstellt.


In [ ]:
/// Unser DatenType für die beiden logischen Operatoren && und ||
#[derive(Debug, Clone)]
pub enum LogicOp {
    And(Expression, Expression),
    Or(Expression, Expression),
}

/// Unser DatenTyp für Vergleiche < > <= >= == und !=
#[derive(Debug, Clone)]
pub enum Comparison {
    Smaller(Expression, Expression),
    SmallerEquals(Expression, Expression),
    Equals(Expression, Expression),
    Greater(Expression, Expression),
    GreaterEquals(Expression, Expression),
}

### Binäre (numerische) Operationen

Um die Grundoperationen darzustellen, verwenden wir einen Struct-Type-Enum. Wir werden dann den Unterschied im Pattern-Matching erkennen können, und was für Vor-/Nachteile es mit sich bringt.

Auch hier fehlt unsere letzte Variant, die Pow-Variant. Füge sie hinzu.

In [ ]:
#[derive(Debug, Clone)]
pub enum BinaryOp {
    Plus { left: Expression, right: Expression },
    Minus { left: Expression, right: Expression },
    Mul { left: Expression, right: Expression },
    Div { left: Expression, right: Expression },
    Mod { left: Expression, right: Expression },
    /* Füge hier den Pow-Variant ein */
    None,
}

### Conditional Expression

In unserer Sprache wird es eine If-Expression haben. Die If-Expression returned jeweils die letzte Expression als Wert, und braucht entsprechend zwingend einen Else-Branch.

In unserem AST stellen wir diese If-Expression als Struct dar. Eine Struct in Rust ist ähnlich einer Klasse in anderen Sprachen. Sie wird mit dem Keyword `struct` definiert. Ähnlich wie in Kotlin werden anschliessend die Feldernamen und der entsprechende Typ aufgezählt. Eine Struct in Rust besitzt allerdings an sich keine Funktionen und entsprechend gibt es keine Möglichkeit Default-Values anzugeben (es gibt allerdings den "Default"-Trait, auf den wir hier aber nicht weiter eingehen).

Eine Struct kann anschliessend mit dem Struct-Initializer-Syntax erstellt werden. Das kleine Beispiel-Listing kannst du ausführen und damit rumspielen.

In [18]:
#[derive(Debug)]
struct DiesIstEineStruct {
    sie_hält_einen_string: String,
    und_einen_integer: u32
}

// erstellen kann man sie nun wie folgt:

println!("{:?}", 
    DiesIstEineStruct {
        sie_hält_einen_string: "Dies ist ein string".into(),
        und_einen_integer: 3
    });


DiesIstEineStruct { sie_hält_einen_string: "Dies ist ein string", und_einen_integer: 3 }


In [2]:
#[derive(Debug, Clone)]
pub struct ConditionalExpression {
    pub condition: Box<Expression>,
    pub body: Vec<Token>,
    pub body_expression: Box<Expression>,
    /* Fügen jeweils else_body und else_body_expression wie für den true branch ein */
}

Error: cannot find type `Expression` in this scope

Error: cannot find type `Token` in this scope

Error: cannot find type `Expression` in this scope

### Heap und Stack

In Rust wird versucht möglichst viel auf dem Stack zu stellen. Wer sich mit Memory-Layout etwas auskennt, wird wissen, dass, um ein Objekt auf dem Stack zu plazieren, die grösse während dem Kompilieren bekannt sein muss. Für die meisten "Grund-Datentypen" ist dies erfüllt. Wir bekommen allerdings ein Problem, sobald wir eine Struktur haben, die sich selbst wieder Referenziert. 

Da wir mit nested Expressions arbeiten werden, z.B. `(3+2+3) * (4+4+6)`, was eine Expression ist die aus zwei (resp. mehr) SubExpressions besteht, führt dies unweigerlich zum Problem, dass die Grösse des Datentyps prinzipiel nicht beschränkt ist. Wir können also im Voraus nicht wissen, wieviele dieser Sub-Expressions auf den Stack müssen, und darum können wir den benötigten Speicher nicht bereitstellen.

Einen Ausweg gibt es, sobald wir Heap-Allozierte Daten-Typen haben. Dies sind meist dynamisch wachsende Typen, wie z.B. Arrays, Maps, Strings (oder eben selbstreferenzierende Strukturen). Bei einem Heap-Alloziertem Typen müssen wir nicht mehr die Struktur auf dem Stack speichern, sondern nur noch eine Referenz auf den entsprechenden Bereich im Heap. Dies löst unser Probelm, mit dem Nachteil, dass wir an "Geschwindigkeit" verlieren, da Zugriffe auf den Heap teurer sind als solche, die nur auf den Stack verweisen.
### Box
Im Beispiel oben verwenden wir den Datentyp `Box<T>`, der einen generischen Typen darstellt, welcher den Datentypen `T` auf dem Heap alloziert, und als Pointer interpretiert wird. Allozierung und Deallozierung werden automatisch von Rust gehandelt, so dass der Speicher freigegeben wird, sobald die `Box<T>` aus dem Scope verschwindet.

Vergleiche nachfolgend die Speichergrösse für einen Boxed Typ und den Typen auf dem Stack. Wir sehen dass der Typ `SizeOfStruct` nur einen Integer hält (4 Bytes) und entsprechend auch nur 4 Bytes gross ist. Der Boxed Type ist abhängig von der zugrundeliegenden Plattform, heutzutage in der Regel 64bits sprich 8 Bytes.

In [21]:
struct SizeOfStruct {
    test: u32
}
println!("{}",std::mem::size_of::<SizeOfStruct>());
println!("{}", std::mem::size_of::<Box<SizeOfStruct>>())

4
8


()

### Arrays (Vec)

Ein weiterer wichtiger Datentyp stellen Arrays dar. Die einfachste Art einen Array in Rust abzubilden ist mit dem `Vec<T>` Datentyp. Dieser ist wiederum generisch zum Typen T. Wie bereits erwähnt wird ein Vec stets auf dem Heap alloziert. Da der Vector nur einem Typen `T` zu geordnet werden kann, muss in bestimmten Fällen der Typ explizit mit angegeben werden. Dies ist i.d.R. der Fall, wenn der Type-Inferer es nicht schafft, einen eindeutigen Typen zu finden. Meist passiert dies mit den `collect` Funktionen, welche auf Iterators definiert sind.

In solchen Fällen kann entweder der Typ direkt bei der Variablen-Deklaration angegeben werden, oder aber mit dem sogenannten Turbo-Fish.

Beachte nun zum Beispiel die zwei Vektoren, und anschliessend, der Gebrauch des Turbo-Fischs bei der `parse`-Funktion.

In [25]:
let array : Vec<u32> = vec![];
let some_elements = vec![0;3];

println!("{:?}", array);
println!("{:?}", some_elements);
println!("{}", "3.2".parse::<f32>().unwrap());

[]
[0, 0, 0]
3.2


### Options

In Rust sind alle Typen immer initialisiert (sprich niemals Null). Es gibt aber Situationen, in denen ein Wert vorhanden, oder eben nicht vorhanden sein kann oder muss.

Falls das ein Resultat einer Operation ist, kann man z.B. den `Result<T>` Datentypen verwenden, den wir später noch kennenlernen werden. Für Structs oder Enums empfiehlt sich der `Option<T>` Datentyp, der im Wesentlichen folgendes Enum darstellt (natürlich noch erweitert mit vielen hilfreichen Funktionen aus der Standardbibliothek).

In [6]:
#[derive(Debug)]
enum CustomOption<T> {
    Some(T),
    None
}

println!("{:?}", CustomOption::Some(0u32));
println!("{:?}", CustomOption::<u32>::None);


Some(0)
None


Unser erster Schritt, das Definieren des ASTs, ist nun abgeschlossen und im folgenden sind die oben erwähnten Strukturen ergänzt mit den Statement Strukturen.

Gehe nochmals durch den Code, und führe ihn anschliessend aus.

In [7]:
#[derive(Debug, Clone)]
pub enum Atomic {
    String(String),
    Number(f64),
    Bool(bool),
    Null,
    Interrupt
}

#[derive(Debug, Clone)]
pub enum LogicOp {
    And(Expression, Expression),
    Or(Expression, Expression),
}


#[derive(Debug, Clone)]
pub enum Comparison {
    Smaller(Expression, Expression),
    SmallerEquals(Expression, Expression),
    Equals(Expression, Expression),
    Greater(Expression, Expression),
    GreaterEquals(Expression, Expression),
}

#[derive(Debug, Clone)]
pub enum BinaryOp {
    Plus { left: Expression, right: Expression },
    Minus { left: Expression, right: Expression },
    Mul { left: Expression, right: Expression },
    Div { left: Expression, right: Expression },
    Mod { left: Expression, right: Expression },
    Pow { left: Expression, right: Expression },
    None,
}

/// Expressions
#[derive(Debug, Clone)]
pub struct ConditionalExpression {
    pub condition: Box<Expression>,
    pub body: Vec<Token>,
    pub body_expression: Box<Expression>,
    pub else_body: Vec<Token>,
    pub else_body_expression: Box<Expression>,
}


#[derive(Debug, Clone)]
pub enum Cast {
    String(Expression),
    Int(Expression),
    Bool(Expression),
}

#[derive(Debug, Clone)]
pub enum Expression {
    Atomic(Atomic),
    Ident(String),
    LogicOp(Box<LogicOp>),
    Comparison(Box<Comparison>),
    BinaryOp(Box<BinaryOp>),
    Conditional(ConditionalExpression),
    Input(Box<Expression>),
    Cast(Box<Cast>),
}

#[derive(Debug, Clone)]
pub enum Statement {
    VariableAssignment(VariableAssignment),
    Conditional(Conditional),
    Expression(Expression),
    Print(Expression),
    Loop(Loop)
}

#[derive(Debug, Clone)]
pub struct Loop {
    pub condition: Box<Expression>,
    pub body: Vec<Token>,
}

#[derive(Debug, Clone)]
pub struct Conditional {
    pub condition: Box<Expression>,
    pub body: Vec<Token>,
    pub else_body: Option<Vec<Token>>,
}

#[derive(Debug, Clone)]
pub struct VariableAssignment {
    pub new_definition: bool,
    pub ident: String,
    pub value: Expression,
}

#[derive(Debug, Clone)]
/// Unsere Sprache besteht aus verschiedenen Tokens
pub enum Token {
    /// Eine Expression ist ein Token, das einen Wert darstellt. Enstprechend kann eine Expression z.B. einer Variable 
    /// hinzugefügt werden.
    Expression(Expression),
    /// Ein Statement führt Code aus, stellt aber keinen Wert dar und kann somit nur alleine stehen
    Statement(Statement),
    /// Vorzeitiges Ende aus einer While-Loop
    Break,
    /// Placeholder für potenzielles early return
    Return
}

## AST Erstellen (Step 2)